# Nuke button

In [ ]:
# Delete all files and folders in the working directory. Use with caution!
import shutil
shutil.rmtree("/kaggle/working", ignore_errors=True)

# Code

## Git Clone

In [ ]:
!git clone -q https://github.com/BinhPQ2/magi_functional.git

In [ ]:
!pip install -qr /kaggle/working/magi_functional/requirements_kaggle.txt

## Load MAGI model

In [ ]:
from PIL import Image
import numpy as np
from transformers import AutoModel
import torch

model = AutoModel.from_pretrained("ragavsachdeva/magiv2", trust_remote_code=True).cuda().eval()

## Save model to local

In [ ]:
# Save model to directory:
model.save_pretrained("./magi_saved_model_directory/")

In [ ]:
# Load model from directory:
model = AutoModel.from_pretrained("./magi_saved_model_directory/", trust_remote_code=True).cuda().eval()

In [ ]:
####################### OR ###########################
# Cache model to directory:
# model_1 = AutoModel.from_pretrained("ragavsachdeva/magiv2", cache_dir="./magi_saved_model_directory/", trust_remote_code=True).cuda().eval()

# Code

## Create raw and character/names list

In [ ]:
import os
import re
from pathlib import Path

def create_chapter_pages_and_character_bank(manga_folder, character_folder):
    # Create lists for chapter pages and character bank
    chapter_pages = []
    character_bank = {
        "images": [],
        "names": []
    }

#     Iterate through manga images to create chapter_pages
    for image_file in os.listdir(manga_folder):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Extract the page number using regex
            match = re.search(r'p(\d+)', image_file)
            if match:
                page_number = int(match.group(1))  # Convert to integer for sorting
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)
            else:
                page_number = image_file.rsplit(".", 1)[0]
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)

    # Sort chapter pages by page number
    chapter_pages.sort(key=lambda x: x[0])
    chapter_pages = [os.path.join(manga_folder, img[1]) for img in chapter_pages]  # Extract just the filenames after sorting

    # Iterate through character images to create character bank
    for char_image_file in os.listdir(character_folder):
        if char_image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Split the filename to extract character name
            char_name = char_image_file.split('_')[0]  # Get the part before the underscore
            character_bank["images"].append(os.path.join(character_folder, char_image_file))
            character_bank["names"].append(char_name)
    return chapter_pages, character_bank

# Define your folders
# One Piece
# manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/raw_manga")
# character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/character")

# Ruri Dragon
manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/raw_manga")
character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/Ruri_Dragon/character")

# Get chapter pages and character bank
chapter_pages_original, character_bank_original = create_chapter_pages_and_character_bank(manga_folder, character_folder)

chapter_pages_test = chapter_pages_original[1:19]
character_bank_test = character_bank_original[:]

# Print the results (for debugging)
print("Chapter Pages:")
print(chapter_pages_test)

print("\nCharacter Bank:")
print(character_bank_test)

## Process (OCR → Transcript)

In [ ]:
import os
import json

def read_image(path_to_image):
    with open(path_to_image, "rb") as file:
        image = Image.open(file).convert("L").convert("RGB")
        image = np.array(image)
    return image

# chapter_pages = ["page1.png", "page2.png", "page3.png" ...]
# character_bank = {
#     "images": ["char1.png", "char2.png", "char3.png", "char4.png" ...],
#     "names": ["Luffy", "Sanji", "Zoro", "Ussop" ...]
# }


chapter_pages = [read_image(x) for x in chapter_pages_test]
character_bank = character_bank_test.copy()
character_bank["images"] = [read_image(x) for x in character_bank_test["images"]]

with torch.no_grad():
    per_page_results = model.do_chapter_wide_prediction(chapter_pages, character_bank, use_tqdm=True, do_ocr=True)

json_output_dir = "/kaggle/working/json_results"
result_image_output_dir = "/kaggle/working/image_results"
os.makedirs(json_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  
os.makedirs(result_image_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  

transcript = []
for i, (image, page_result) in enumerate(zip(chapter_pages, per_page_results)):
    model.visualise_single_image_prediction(image, page_result, os.path.join(result_image_output_dir, f"page_{i}.png"))
    # Save page_result to JSON
    json_file_path = os.path.join(json_output_dir, f"page_{i}_result.json")  # Create full file path
    with open(json_file_path, 'w') as json_file:
        json.dump(page_result, json_file, indent=4)  # Save with pretty printing

    speaker_name = {
        text_idx: page_result["character_names"][char_idx] for text_idx, char_idx in page_result["text_character_associations"]
    }
    for j in range(len(page_result["ocr"])):
        if not page_result["is_essential_text"][j]:
            continue
        name = speaker_name.get(j, "unsure") 
        transcript.append(f"<{name}>: {page_result['ocr'][j]}")
    transcript.append("---------------------------------------------")
with open(f"transcript.txt", "w") as fh:
    for line in transcript:
        fh.write(line + "\n")

print("\n\nDone you WEEEEB!")

# Zip file

In [ ]:
json_output_dir = "/kaggle/working/json_results"
result_image_output_dir = "/kaggle/working/image_results"

In [ ]:
# zip
json_output_dir = "/kaggle/working/json_results"
!zip -rj /kaggle/working/json.zip {json_output_dir}/*.json
# download
!cd /kaggle/working/
display(FileLink('json.zip'))

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("json_results", f"{json_output_dir}/*.json")

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("image_results", result_image_output_dir)

In [ ]:
!ls -la {json_output_dir}